# Benchmark DNNCO G Only

> 

In [ ]:
import os, json, re

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna  import *
from EnvDL.dlfn import * # includes LSUV_

from tqdm import tqdm

In [ ]:
use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F # F.mse_loss

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger


device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.10_g2fc_benchmark_DNNCO_G/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_env_lookup  = np.load(load_from+'obs_env_lookup.npy')
obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx	Geno_Idx	Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')
SMat = np.load(load_from+'SMat.npy')
WMat = np.load(load_from+'WMat.npy')
# PlantHarvestNames = np.load(load_from+'PlantHarvestNames.npy') 
PlantHarvest = np.load(load_from+'PlantHarvest.npy')  # 'DOY_Planted', 'DOY_Harvested'

### Screeplot

In [ ]:
load_from = '../data/zma/g2fc/genotypes_G3_filter/'

GScree = pd.read_table(load_from+'Eigenvalues_Imputed_5_Genotype_Data_All_Years_G3_Hz_0.0675.txt',
                     sep = '\t',
                     low_memory = False)
GScree

fig = px.line(GScree, x = 'PC', y = 'cumulative proportion', log_x=True)
pr_bins = [(2+i)/10 for i in range(8)]
for i in range(len(pr_bins)-1):
    x_start = GScree.loc[(GScree['cumulative proportion'] >= pr_bins[i]), 'PC'].min()
    x_stop = GScree.loc[(GScree['cumulative proportion'] >= pr_bins[i+1]), 'PC'].min()
   
    if i%2 == 1:
        fig.add_vrect(x0=x_start, 
                      x1=x_stop, 
                      fillcolor="green", opacity=0.25, line_width=0)
    else:
        fig.add_vrect(x0=x_start, 
                      x1=x_stop, 
                      fillcolor="white", opacity=0.25, line_width=0)
fig.show()

### PCA Transformed Data

In [ ]:
GMat = pd.read_table(load_from+'PC_Imputed_5_Genotype_Data_All_Years_G3_Hz_0.0675.txt',
                     sep = '\t',
                     skiprows=2,
                    low_memory = False)
GMat.shape # two more entries than the data I've been using.

(4928, 4882)

In [ ]:
# need to check taxa match
assert 0 == len([e for e in list(set(phno_geno['Hybrid'])) if e not in list(GMat['Taxa'])])

In [ ]:
# Get the desired genotypes in the desired order
temp = pd.concat([phno_geno['Hybrid'], pd.DataFrame(obs_geno_lookup)], axis = 1)
temp = temp.loc[:, ['Hybrid', 1]].drop_duplicates().sort_values(1).reset_index(drop = True)
temp

,Hybrid,1
0,M0088/LH185,0
1,M0143/LH185,1
2,M0003/LH185,2
3,M0035/LH185,3
4,M0052/LH185,4
...,...,...
4921,GEMN-0096_LH212HT_0014/LH244,4921
4922,NC370/LH244,4922
4923,GEMN-0097_LH212HT_0046/LH244,4923
4924,GEMN-0225_PHK76_0025/LH244,4924


In [ ]:
# Reorder PCA data, then convert to numpy
GMat = temp.merge(GMat.rename(columns = {'Taxa':'Hybrid', 'index':'Gpca_Idx'})).drop(columns = 1)
GMatHybrid = GMat['Hybrid']
GMat = GMat.loc[:, [i for i in list(GMat) if i != 'Hybrid']]

In [ ]:
GMat = np.asarray(GMat)

In [ ]:
# We should now be able to use obs_geno_lookup to lookup PCA values.
assert GMat.shape[0] == len(set(obs_geno_lookup[:, 1]))

### Custom Dataloader for G S W

In [ ]:
window_start = PlantHarvest[:, 0]-76 # 75 before planting date
window_end   = PlantHarvest[:, 0]+212 # 212 after (288 total per W.npy in paper)
window_end - window_start

array([288, 288, 288, ..., 288, 288, 288])

In [ ]:
# the increased range of planting dates causes the window to extend beyond the target year. 
(window_start.min(), 
 window_end.max() - 364) #pd.to_datetime() is 0 indexed

(-16, 18)

In [ ]:
window_start = PlantHarvest[:, 0]-60
window_end   = PlantHarvest[:, 0]+194
window_end - window_start

array([254, 254, 254, ..., 254, 254, 254])

In [ ]:
assert window_start.min() >= 0 

In [ ]:
assert window_end.max() <= 364 #pd.to_datetime() is 0 indexed

In [ ]:
class GSWDataset(Dataset): # for any G containing matix with many (phno) to one (geno)
    def __init__(self, 
                 y, 
                 G, # not on gpu
                 S,
                 W,
                 
                 planting, # should be the vector so [:, 0]
                 days_before_planting,
                 days_after_planting,
                 
                 idx_original,
                 idx_lookup_geno,
                 idx_lookup_env,
                 
                 transform = None, target_transform = None,
                 device = 'cuda',
                 **kwargs 
                ):

        self.device = device
        self.y = y 
        self.G = G
        self.S = S
        self.W = W
        
        self.window_start = planting - days_before_planting
        self.window_end   = planting + days_after_planting
        
        self.idx_original = idx_original
        self.idx_lookup_geno = idx_lookup_geno
        self.idx_lookup_env  = idx_lookup_env
        
        self.transform = transform
        self.target_transform = target_transform    
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        y_idx =self.y[idx]
            
        #                 |array containing correct index in deduplicated g 
        #                 |               index in phno    
        uniq_g_idx = self.idx_lookup_geno[
            self.idx_original[idx], 1]
        g_idx = self.G[uniq_g_idx, :]
        
        
        #                 |array containing correct index in deduplicated w 
        #                 |               index in phno    
        uniq_env_idx = self.idx_lookup_env[self.idx_original[idx], 1]
        s_idx = self.S[uniq_env_idx, :]
        w_idx = self.W[uniq_env_idx, :, self.window_start[idx]:self.window_end[idx]]
        
        # send all to gpu        
        if (self.device != 'cpu'):
            if y_idx.device.type == 'cpu':
                y_idx = y_idx.to(self.device) 
                
            if g_idx.device.type == 'cpu':
                g_idx = g_idx.to(self.device) 
                
            if s_idx.device.type == 'cpu':
                s_idx = s_idx.to(self.device)                 
                
            if w_idx.device.type == 'cpu':
                w_idx = w_idx.to(self.device)         
        
        if self.transform:
            y_idx = self.transform(y_idx)
            g_idx = self.transform(g_idx)
            s_idx = self.transform(s_idx)
            w_idx = self.transform(w_idx)
            
        return y_idx, g_idx, s_idx, w_idx

### Create train/test validate indicies from json

In [ ]:
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

In [ ]:
split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

In [ ]:
temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

In [ ]:
test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)
# test_dict

In [ ]:
# since this is applying predefined model structure no need for validation.
# This is included for my future reference when validation is needed.
temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)
# val_dict

In [ ]:
test_dict

{'test_idx': Index([103326, 103331, 103352, 103362, 103390, 103393, 103405, 103408, 103416,
        103428,
        ...
        133044, 133045, 133046, 133047, 133048, 133050, 133051, 133052, 133053,
        133056],
       dtype='int64', length=9412),
 'train_idx': Index([     0,      1,      4,      6,      7,      9,     10,     11,     13,
            14,
        ...
        103296, 103297, 103298, 103299, 103300, 103301, 103302, 103303, 103304,
        103305],
       dtype='int64', length=81169)}

In [ ]:
# validation not strictly needed since there is no tuning to do. Included to help with comparison to other models
# train_idx = test_dict['train_idx']
# test_idx  = test_dict['test_idx']

train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
# confirm all observation idxs are have genomic information
assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
SMat_cs = calc_cs(SMat[list(set(obs_env_lookup[train_idx, 1])), ])
s_cs = apply_cs(SMat, SMat_cs)

WMat_cs = calc_cs(WMat[list(set(obs_env_lookup[train_idx, 1])), ])
w_cs = apply_cs(WMat, WMat_cs)

In [ ]:
GMat_cs = calc_cs(GMat[list(set(obs_geno_lookup[train_idx, 1])), ])
g_cs = apply_cs(GMat, GMat_cs)

In [ ]:
training_dataloader = DataLoader(
    GSWDataset(
        y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
        G = torch.from_numpy(g_cs).to(torch.float32), 
        S = torch.from_numpy(s_cs).to(torch.float32),
        W = torch.from_numpy(w_cs).to(torch.float32),
        planting = PlantHarvest[:, 0],
        days_before_planting = 60,
        days_after_planting = 194,
        idx_original = torch.from_numpy(np.array(train_idx)),#[:, None],
        idx_lookup_geno = obs_geno_lookup,
        idx_lookup_env = obs_env_lookup,
        device = 'cuda'),
    batch_size = 256,
    shuffle = True
)
# next(iter(training_dataloader))

testing_dataloader = DataLoader(
    GSWDataset(
        y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
        G = torch.from_numpy(g_cs).to(torch.float32), 
        S = torch.from_numpy(s_cs).to(torch.float32),
        W = torch.from_numpy(w_cs).to(torch.float32),
        planting = PlantHarvest[:, 0],
        days_before_planting = 60,
        days_after_planting = 194,
        idx_original = torch.from_numpy(np.array(test_idx)),#[:, None],
        idx_lookup_geno = obs_geno_lookup,
        idx_lookup_env = obs_env_lookup,
        device = 'cuda'),
    batch_size = 256,
    shuffle = True
)
# next(iter(testing_dataloader))

In [ ]:
# y_idx, g_idx, s_idx, w_idx = next(iter(training_dataloader))

## Use Pytorch Lightning to Train each Model

In [ ]:
# def Linear_block(in_size, out_size, drop_pr):
#     block = nn.Sequential(
#         nn.Linear(in_size, out_size),
#         nn.ReLU(),
#         nn.Dropout(drop_pr)
#     )
#     return(block) 

# def Conv1D_x2_Max_block(in_channels, out_channels, kernel_size, stride, maxpool_size):
#     block = nn.Sequential(
#         nn.Conv1d(
#             in_channels= in_channels, # second channel
#             out_channels= out_channels,
#             kernel_size= kernel_size,
#             stride= stride
#         ), 
#         nn.Conv1d(
#             in_channels= out_channels, 
#             out_channels= out_channels,
#             kernel_size= kernel_size,
#             stride= stride
#         ), 
#         nn.BatchNorm1d(out_channels),
#         nn.MaxPool1d((maxpool_size), stride=stride)
#     )
#     return(block)

In [ ]:
class DNNCO_G(nn.Module):
    def __init__(self):
        super(DNNCO_G, self).__init__()    
        
        self.g_network =nn.Sequential(
            Linear_block(in_size= 4881, out_size= 83, drop_pr= 0.163923177),
            Linear_block(in_size= 83,   out_size= 133, drop_pr= 0.230663142)
        ) 
        self.g_pred =nn.Sequential(
            nn.Linear(133, 1)
        ) 
        
        
    def forward(self, g, s, w):
        g_out = self.g_network(g)
        g_pred = self.g_pred(g_out)
        return g_pred, g_out
    
# [e.shape for e in DNNCO_G().to(device)(g_idx, s_idx, w_idx)]    

In [ ]:
class DNNCO_S(nn.Module):
    def __init__(self):
        super(DNNCO_S, self).__init__()    
        
        self.s_network =nn.Sequential(
            Linear_block(in_size= 23, out_size= 38, drop_pr= 0.148724301),
            Linear_block(in_size= 38, out_size= 13, drop_pr= 0.276340999),
            Linear_block(in_size= 13, out_size= 45, drop_pr= 0.005434164),
            Linear_block(in_size= 45, out_size= 29, drop_pr= 0.173380695),
            Linear_block(in_size= 29, out_size= 4,  drop_pr= 0.),
            Linear_block(in_size= 4,  out_size= 4,  drop_pr= 0.),
            Linear_block(in_size= 4,  out_size= 4,  drop_pr= 0.)
        )  
        self.s_pred =nn.Sequential(
            nn.Linear(4, 1)
        ) 
        
    def forward(self, g, s, w):
        s_out = self.s_network(s)
        s_pred = self.s_pred(s_out)
        return s_pred, s_out
    
# [e.shape for e in DNNCO_S().to(device)(g_idx, s_idx, w_idx)]

In [ ]:
class DNNCO_W(nn.Module):
    def __init__(self):
        super(DNNCO_W, self).__init__()  
        
        self.w_network =nn.Sequential(
            Conv1D_x2_Max_block(
                in_channels = 16, 
                out_channels = 433, 
                kernel_size = 3,
                maxpool_size = 2,
                stride = 1),
            Conv1D_x2_Max_block(
                in_channels = 433, 
                out_channels = 436, 
                kernel_size = 3,
                maxpool_size = 2,
                stride = 1),
            Conv1D_x2_Max_block(
                in_channels = 436, 
                out_channels = 52, 
                kernel_size = 3,
                maxpool_size = 2,
                stride = 1),
            Conv1D_x2_Max_block(
                in_channels = 52, 
                out_channels = 163, 
                kernel_size = 3,
                maxpool_size = 2,
                stride = 1),
            Conv1D_x2_Max_block(
                in_channels = 163, 
                out_channels = 400, 
                kernel_size = 3,
                maxpool_size = 2,
                stride = 1),
            Conv1D_x2_Max_block(
                in_channels = 400, 
                out_channels = 294, 
                kernel_size = 3,
                maxpool_size = 2,
                stride = 1)
        )
        
        self.w_flatten =nn.Flatten()

        self.w_pred =nn.Sequential(
            nn.Linear(65856, 1)
        )
        
    def forward(self, g, s, w):
        w_out = self.w_network(w)
        w_out = self.w_flatten(w_out)
        w_pred = self.w_pred(w_out)
        return w_pred, w_out
    
# [e.shape for e in DNNCO_W().to(device)(g_idx, s_idx, w_idx)]

In [ ]:
# [[e.shape for e in DNNCO_G().to(device)(g_idx, s_idx, w_idx)],
#  [e.shape for e in DNNCO_S().to(device)(g_idx, s_idx, w_idx)],
#  [e.shape for e in DNNCO_W().to(device)(g_idx, s_idx, w_idx)]]

In [ ]:
class DNNCO_X(nn.Module):
    def __init__(self):
        super(DNNCO_X, self).__init__()  
        
        self.x_network =nn.Sequential(
            Linear_block(in_size= 65993, out_size= 152, drop_pr= 0.18658661),
            Linear_block(in_size= 152,   out_size= 207, drop_pr= 0.289893588),
            Linear_block(in_size= 207,   out_size= 206, drop_pr= 0.004841293),
            Linear_block(in_size= 206,   out_size= 188, drop_pr= 0.198121953),
            Linear_block(in_size= 188,   out_size= 44,  drop_pr= 0.243027717),
            Linear_block(in_size= 44,    out_size= 1,   drop_pr= 0.0)
        )  
        
    def forward(self, g, s, w):        
        x_pred = self.x_network(torch.concat([g, s, w], axis = 1))
        return x_pred, None
    
# DNNCO_X().to(device)(
#     DNNCO_G().to(device)(g_idx, s_idx, w_idx)[1],
#     DNNCO_S().to(device)(g_idx, s_idx, w_idx)[1],
#     DNNCO_W().to(device)(g_idx, s_idx, w_idx)[1])[0].shape   

In [ ]:
# Module for training subnetworks.
class plDNNCO_subnet(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        # train loop
        y_i, g_i, s_i, w_i = batch
        pred, out = self.mod(g_i, s_i, w_i)
        # print(y_i.shape, pred.shape)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        # calculate layerwise stats 
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        y_i, g_i, s_i, w_i = batch
        pred, out = self.mod(g_i, s_i, w_i)
        # print(y_i.shape, pred.shape)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    
    
    
# DNNG = plDNNCO_G(DNNCO_G().to(device))

# optimizer = DNNG.configure_optimizers()

# for batch_idx, batch in tqdm(enumerate(training_dataloader)):
#     loss = DNNG.training_step(batch, batch_idx)
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()    

In [ ]:
# Module for training Interaction networks. 
# This version uses the subnetworks so it would be good for predicting new data but this could be
# reformulated to instead use predictions from the subnetworks which would be more preformant.
class plDNNCO_X(pl.LightningModule):
    def __init__(self, mod, Gmod, Smod, Wmod):
        super().__init__()
        self.mod = mod
        self.Gmod = Gmod
        self.Smod = Smod
        self.Wmod = Wmod
    def training_step(self, batch, batch_idx):
        # train loop
        y_i, g_i, s_i, w_i = batch
        
        with torch.no_grad():
            _, g_out = self.Gmod(g_i, s_i, w_i)            
            _, s_out = self.Smod(g_i, s_i, w_i)            
            _, w_out = self.Wmod(g_i, s_i, w_i)            
        
        pred, out = self.mod(g_out, s_out, w_out)
        # print(y_i.shape, pred.shape)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        return(loss)
    
    def configure_optimizers(self, **kwargs):
        optimizer =  torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer
    

### Train with logger

In [ ]:
# Table3 in G3 paper https://academic.oup.com/g3journal/article/13/4/jkad006/6982634
adam_params ={
          #lr,     beta1,    beta2,    batch_size, max_epoch
    'g':  [0.0001, 0.953368, 0.985947,  96,  12 ],
    's':  [0.01,   0.928472, 0.997516, 176, 199],
    'w':  [0.0001, 0.903649, 0.929582, 240, 629],
    'co': [0.01,   0.98752,  0.972311, 112, 364],
    'so': [0.001,  0.975893, 0.994607, 192, 711]}

In [ ]:
subnet = 'g'                                                         # 0. Update
lr, beta1, beta2, batch_size, max_epoch = adam_params[subnet]
# # max_epoch = 2 #FIXME
# training_dataloader = DataLoader(
#     GSWDataset(
#         y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
#         G = torch.from_numpy(g_cs).to(torch.float32), 
#         S = torch.from_numpy(s_cs).to(torch.float32),
#         W = torch.from_numpy(w_cs).to(torch.float32),
#         planting = PlantHarvest[:, 0],
#         days_before_planting = 60,
#         days_after_planting = 194,
#         idx_original = torch.from_numpy(np.array(train_idx)),#[:, None],
#         idx_lookup_geno = obs_geno_lookup,
#         idx_lookup_env = obs_env_lookup,
#         device = 'cuda'),
#     batch_size = batch_size,
#     shuffle = True
# )

# validation_dataloader = DataLoader(
#     GSWDataset(
#         y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
#         G = torch.from_numpy(g_cs).to(torch.float32), 
#         S = torch.from_numpy(s_cs).to(torch.float32),
#         W = torch.from_numpy(w_cs).to(torch.float32),
#         planting = PlantHarvest[:, 0],
#         days_before_planting = 60,
#         days_after_planting = 194,
#         idx_original = torch.from_numpy(np.array(test_idx)),#[:, None],
#         idx_lookup_geno = obs_geno_lookup,
#         idx_lookup_env = obs_env_lookup,
#         device = 'cuda'),
#     batch_size = batch_size,
#     shuffle = True
# )

In [ ]:
# DNNG = plDNNCO_subnet(DNNCO_G().to(device))                          # 1. Update
# optimizer = DNNG.configure_optimizers(lr = lr, betas=(beta1, beta2)) # 2. Update

# logger = TensorBoardLogger("tb_logs", name="dnnco-g")                # 3. Update
# trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# trainer.fit(model=DNNG, 
#             train_dataloaders=training_dataloader, val_dataloaders= validation_dataloader)       # 4. Update

In [ ]:
# As currently formulated LSUV_ only works if a single data object is passed in.
# options are to 
# 1. rewrite dataloader with a dict of data
# 2. rewrite LSUV_
# 3. use kaiming and come back to this.
# LSUV_(model, data = next(iter(training_dataloader))[1:])

# model = DNNCO_G().to(device)

# def init_weigths(m):
#     if isinstance(m, (nn.Linear)): nn.init.kaiming_normal_(m.weight)
        
# model = model.apply(init_weigths)

# DNNG = plDNNCO_subnet(model)                          # 1. Update
# optimizer = DNNG.configure_optimizers(lr = lr, betas=(beta1, beta2)) # 2. Update

# logger = TensorBoardLogger("tb_logs", name="dnnco-g-k")                # 3. Update
# trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# trainer.fit(model=DNNG, train_dataloaders=training_dataloader,val_dataloaders= validation_dataloader)       # 4. Update

In [ ]:
# LSUV compatable version

class LGSWDataset(Dataset): # for any G containing matix with many (phno) to one (geno)
    def __init__(self, 

                 y, 
                 G, # not on gpu
                 S,
                 W,

                 planting, # should be the vector so [:, 0]
                 days_before_planting,
                 days_after_planting,

                 idx_original,
                 idx_lookup_geno,
                 idx_lookup_env,

                 transform = None, target_transform = None,
                 device = 'cuda',
                 **kwargs 
                ):

        self.device = device
        self.y = y 
        self.G = G
        self.S = S
        self.W = W

        self.window_start = planting - days_before_planting
        self.window_end   = planting + days_after_planting

        self.idx_original = idx_original
        self.idx_lookup_geno = idx_lookup_geno
        self.idx_lookup_env  = idx_lookup_env

        self.transform = transform
        self.target_transform = target_transform    

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        y_idx =self.y[idx]

        #                 |array containing correct index in deduplicated g 
        #                 |               index in phno    
        uniq_g_idx = self.idx_lookup_geno[
            self.idx_original[idx], 1]
        g_idx = self.G[uniq_g_idx, :]


        #                 |array containing correct index in deduplicated w 
        #                 |               index in phno    
        uniq_env_idx = self.idx_lookup_env[self.idx_original[idx], 1]
        s_idx = self.S[uniq_env_idx, :]
        w_idx = self.W[uniq_env_idx, :, self.window_start[idx]:self.window_end[idx]]

        # send all to gpu        
        if (self.device != 'cpu'):
            if y_idx.device.type == 'cpu':
                y_idx = y_idx.to(self.device) 

            if g_idx.device.type == 'cpu':
                g_idx = g_idx.to(self.device) 

            if s_idx.device.type == 'cpu':
                s_idx = s_idx.to(self.device)                 

            if w_idx.device.type == 'cpu':
                w_idx = w_idx.to(self.device)         

        if self.transform:
            y_idx = self.transform(y_idx)
            g_idx = self.transform(g_idx)
            s_idx = self.transform(s_idx)
            w_idx = self.transform(w_idx)

#         res_list = [[y_idx, g_idx, s_idx, w_idx]]
        res_list = [y_idx, g_idx, s_idx, w_idx]
#         return list(res_list)
        return res_list
    
training_dataloader = DataLoader(
    LGSWDataset(
        y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
        G = torch.from_numpy(g_cs).to(torch.float32), 
        S = torch.from_numpy(s_cs).to(torch.float32),
        W = torch.from_numpy(w_cs).to(torch.float32),
        planting = PlantHarvest[:, 0],
        days_before_planting = 60,
        days_after_planting = 194,
        idx_original = torch.from_numpy(np.array(train_idx)),#[:, None],
        idx_lookup_geno = obs_geno_lookup,
        idx_lookup_env = obs_env_lookup,
        device = 'cuda'),
    batch_size = 256,
    shuffle = True
)

validation_dataloader = DataLoader(
    LGSWDataset(
        y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
        G = torch.from_numpy(g_cs).to(torch.float32), 
        S = torch.from_numpy(s_cs).to(torch.float32),
        W = torch.from_numpy(w_cs).to(torch.float32),
        planting = PlantHarvest[:, 0],
        days_before_planting = 60,
        days_after_planting = 194,
        idx_original = torch.from_numpy(np.array(test_idx)),#[:, None],
        idx_lookup_geno = obs_geno_lookup,
        idx_lookup_env = obs_env_lookup,
        device = 'cuda'),
    batch_size = 256,
    shuffle = True
)

In [ ]:
# [[ee.shape for ee in e] for e in next(iter(training_dataloader))]

class DNNCO_G(nn.Module):
    def __init__(self):
        super(DNNCO_G, self).__init__()    
        
        self.g_network =nn.Sequential(
            Linear_block(in_size= 4881, out_size= 83, drop_pr= 0.163923177),
            Linear_block(in_size= 83,   out_size= 133, drop_pr= 0.230663142)
        ) 
        self.g_pred =nn.Sequential(
            nn.Linear(133, 1)
        ) 
                
    def forward(self, tensor_list):
        y, g, s, w = tensor_list
        g_out = self.g_network(g)
        g_pred = self.g_pred(g_out)
        return g_pred, g_out
    
DNNCO_G().to(device)(next(iter(training_dataloader)) )[0].shape

model = DNNCO_G().to(device)
LSUV_(model, data = next(iter(training_dataloader)) )

Applying orthogonal init (zero init if dim < 2) to params in 3 module(s).
Applying LSUV to 3 module(s) (up to 10 iters per module):
Module  0 after  1 itr(s) | Mean: -0.017 | Std: 0.997 | <class 'torch.nn.modules.linear.Linear'>
Module  1 after  2 itr(s) | Mean: -0.050 | Std: 1.002 | <class 'torch.nn.modules.linear.Linear'>
Module  2 after  2 itr(s) | Mean: -0.504 | Std: 1.098 | <class 'torch.nn.modules.linear.Linear'>


In [ ]:
# Module for training subnetworks.
class plDNNCO_subnet(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        # train loop
#         y_i, g_i, s_i, w_i = batch
#         pred, out = self.mod(g_i, s_i, w_i)
        y_i, _, _, _ = batch
        pred, out = self.mod(batch)
        # print(y_i.shape, pred.shape)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
#         y_i, g_i, s_i, w_i = batch
#         pred, out = self.mod(g_i, s_i, w_i)
        y_i, _, _, _ = batch
        pred, out = self.mod(batch)
        # print(y_i.shape, pred.shape)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
DNNG = plDNNCO_subnet(model)                          # 1. Update
optimizer = DNNG.configure_optimizers(lr = lr, betas=(beta1, beta2)) # 2. Update

logger = TensorBoardLogger("tb_logs", name="dnnco-g-lsuv")                # 3. Update
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)       # 4. Update

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs/dnnco-g-lsuv
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | mod  | DNNCO_G | 416 K 
---------------------------------
416 K     Trainable params
0         Non-trainable params
416 K     Total params
1.666     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: PossibleUserWarning:

Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.



Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=12` reached.


In [ ]:
torch.save(DNNG.mod, cache_path+'dnnco-g-lsuv'+'.pt')

In [ ]:
# subnet = 's'                                                         # 0. Update
# lr, beta1, beta2, batch_size, max_epoch = adam_params[subnet]
# max_epoch = 2 #FIXME
# training_dataloader = DataLoader(
#     GSWDataset(
#         y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
#         G = torch.from_numpy(g_cs).to(torch.float32), 
#         S = torch.from_numpy(s_cs).to(torch.float32),
#         W = torch.from_numpy(w_cs).to(torch.float32),
#         planting = PlantHarvest[:, 0],
#         days_before_planting = 60,
#         days_after_planting = 194,
#         idx_original = torch.from_numpy(np.array(train_idx)),#[:, None],
#         idx_lookup_geno = obs_geno_lookup,
#         idx_lookup_env = obs_env_lookup,
#         device = 'cuda'),
#     batch_size = batch_size,
#     shuffle = True
# )

# DNNS = plDNNCO_subnet(DNNCO_S().to(device))                          # 1. Update
# optimizer = DNNS.configure_optimizers(lr = lr, betas=(beta1, beta2)) # 2. Update

# logger = TensorBoardLogger("tb_logs", name="dnnco-s")                # 3. Update
# trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# trainer.fit(model=DNNS, train_dataloaders=training_dataloader)       # 4. Update

In [ ]:
# subnet = 'w'                                                         # 0. Update
# lr, beta1, beta2, batch_size, max_epoch = adam_params[subnet]
# max_epoch = 2 #FIXME
# training_dataloader = DataLoader(
#     GSWDataset(
#         y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
#         G = torch.from_numpy(g_cs).to(torch.float32), 
#         S = torch.from_numpy(s_cs).to(torch.float32),
#         W = torch.from_numpy(w_cs).to(torch.float32),
#         planting = PlantHarvest[:, 0],
#         days_before_planting = 60,
#         days_after_planting = 194,
#         idx_original = torch.from_numpy(np.array(train_idx)),#[:, None],
#         idx_lookup_geno = obs_geno_lookup,
#         idx_lookup_env = obs_env_lookup,
#         device = 'cuda'),
#     batch_size = batch_size,
#     shuffle = True
# )

# DNNW = plDNNCO_subnet(DNNCO_W().to(device))                          # 1. Update
# optimizer = DNNW.configure_optimizers(lr = lr, betas=(beta1, beta2)) # 2. Update

# logger = TensorBoardLogger("tb_logs", name="dnnco-w")                # 3. Update
# trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# trainer.fit(model=DNNW, train_dataloaders=training_dataloader)       # 4. Update

In [ ]:
# # Now freeze layers to be extra sure they are not updated
# for param in DNNG.mod.parameters():
#     param.requires_grad = False
    
# for param in DNNS.mod.parameters():
#     param.requires_grad = False
    
# for param in DNNW.mod.parameters():
#     param.requires_grad = False

In [ ]:
# DNNS = plDNNCO_subnet(DNNCO_S().to(device))
# optimizer = DNNS.configure_optimizers()
# logger = TensorBoardLogger("tb_logs", name="dnnco-sonly")
# trainer = pl.Trainer(#limit_train_batches=100, 
#                      max_epochs=10, logger=logger)
# trainer.fit(model=DNNS, train_dataloaders=training_dataloader)

In [ ]:
# DNNW = plDNNCO_subnet(DNNCO_W().to(device))
# optimizer = DNNW.configure_optimizers()
# logger = TensorBoardLogger("tb_logs", name="dnnco-wonly")
# trainer = pl.Trainer(#limit_train_batches=100, 
#                      max_epochs=10, logger=logger)
# trainer.fit(model=DNNW, train_dataloaders=training_dataloader)

In [ ]:
# subnet = 'co'                                                        # 0. Update
# lr, beta1, beta2, batch_size, max_epoch = adam_params[subnet]
# max_epoch = 2 #FIXME
# training_dataloader = DataLoader(
#     GSWDataset(
#         y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float32), #torch.from_numpy(YMat[[0, 1], ]), 
#         G = torch.from_numpy(g_cs).to(torch.float32), 
#         S = torch.from_numpy(s_cs).to(torch.float32),
#         W = torch.from_numpy(w_cs).to(torch.float32),
#         planting = PlantHarvest[:, 0],
#         days_before_planting = 60,
#         days_after_planting = 194,
#         idx_original = torch.from_numpy(np.array(train_idx)),#[:, None],
#         idx_lookup_geno = obs_geno_lookup,
#         idx_lookup_env = obs_env_lookup,
#         device = 'cuda'),
#     batch_size = batch_size,
#     shuffle = True
# )

# # DNNW = plDNNCO_subnet(DNNCO_W().to(device))                          # 1. Update
# # optimizer = DNNW.configure_optimizers(lr = lr, betas=(beta1, beta2)) # 2. Update

# # logger = TensorBoardLogger("tb_logs", name="dnnco-w")                # 3. Update
# # trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# # trainer.fit(model=DNNW, train_dataloaders=training_dataloader)       # 4. Update


# DNNX = plDNNCO_X(DNNCO_X().to(device), Gmod = DNNG.mod, Smod = DNNS.mod, Wmod = DNNW.mod)
# optimizer = DNNX.configure_optimizers(lr = lr, betas=(beta1, beta2))

# logger = TensorBoardLogger("tb_logs", name="dnnco-x")
# trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

# trainer.fit(model=DNNX, train_dataloaders=training_dataloader)

In [ ]:
# DNNX = plDNNCO_X(DNNCO_X().to(device), Gmod = DNNG.mod, Smod = DNNS.mod, Wmod = DNNW.mod)
# optimizer = DNNS.configure_optimizers()
# logger = TensorBoardLogger("tb_logs", name="dnnco-x2")
# trainer = pl.Trainer(#limit_train_batches=100, 
#                      max_epochs=10, logger=logger)
# trainer.fit(model=DNNX, train_dataloaders=training_dataloader)

In [ ]:
# from lightning.pytorch import Trainer

# logger = TensorBoardLogger("tb_logs", name="dnnco-gonly")
# # trainer = Trainer(logger=logger)
# # train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
# trainer = pl.Trainer(#limit_train_batches=100, 
#                      max_epochs=10, logger=logger)
# trainer.fit(model=DNNG, train_dataloaders=training_dataloader)